In [1]:
!pip install tensorflow_datasets

  Preparing metadata (setup.py) ... done
  Using cached pyarrow-16.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 7.5 MB/s eta 0:00:00
Using cached pyarrow-16.1.0-cp312-cp312-manylinux_2_28_x86_64.whl (40.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.2/229.2 kB 2.9 MB/s eta 0:00:0000:0100:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21416 sha256=55362eb0b9598c5f2bee129b2f151dd76253557a383a759a09792cab6318090f
  Stored in directory: /home/noah/.cache/pip/wheels/e7/e6/28/864bdfee5339dbd6ddcb5a186286a8e217648ec198bdf0097d
Successfully built promise


In [45]:
import stanza
stanza.download('en')
NLPL = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Downloaded file to /home/noah/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /home/noah/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /home/noah/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /home/noah/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [52]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load and prepare the IMDB dataset
max_features = 10000  # number of words to consider as features
maxlen = 200  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print("Loading data...")
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), "train sequences")
print(len(input_test), "test sequences")

print("Pad sequences (samples x time)")
input_train = pad_sequences(input_train, maxlen=maxlen)
input_test = pad_sequences(input_test, maxlen=maxlen)
print("input_train shape:", input_train.shape)
print("input_test shape:", input_test.shape)

# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
print("Training the model...")
history = model.fit(input_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    validation_data=(input_test, y_test))

# Evaluate the model
score, acc = model.evaluate(input_test, y_test,
                            batch_size=batch_size)

print("Test score:", score)
print("Test accuracy:", acc)


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
input_train shape: (25000, 200)
input_test shape: (25000, 200)
Training the model...
Epoch 1/5


/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 172s 216ms/step - accuracy: 0.7143 - loss: 0.5418 - val_accuracy: 0.8482 - val_loss: 0.3693
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 168s 215ms/step - accuracy: 0.8419 - loss: 0.3696 - val_accuracy: 0.8482 - val_loss: 0.3636
Epoch 3/5
523/782 ━━━━━━━━━━━━━━━━━━━━ 42s 163ms/step - accuracy: 0.8782 - loss: 0.2991

KeyboardInterrupt: 

In [49]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# Load the IMDB dataset word index
word_index = imdb.get_word_index()

# Function to convert a sentence into the required format
def preprocess_sentence(sentence, word_index, maxlen=200):
    # Tokenize the sentence
    tokens = sentence.lower().split()

    # Convert tokens to indices
    indices = [word_index.get(word, 0) for word in tokens]  # Use 0 if word is not found

    # Pad the sequence
    padded_sequence = pad_sequences([indices], maxlen=maxlen)

    return padded_sequence

# Example sentence
sentence = "The Disability Access Center does not have a spine."

# Preprocess the sentence
preprocessed_sentence = preprocess_sentence(sentence, word_index)
# print(preprocessed_sentence)

# Predict sentiment
prediction = model.predict(preprocessed_sentence)

# Output the result
print(f"Prediction: {prediction[0][0]}")
if prediction[0][0] > 0.5:
    print("Sentiment: Positive")
else:
    print("Sentiment: Negative")

2024-06-12 13:47:28.730365: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: indices[0,192] = 10451 is not in [0, 10000)
	 [[{{node sequential_4_1/embedding_1/GatherV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_4_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/noah/Documents/disability_trends/venv/lib64/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib64/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib64/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib64/python3.12/asyncio/events.py", line 88, in _ru

In [50]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
print(sia.polarity_scores(sentence))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [51]:
lemma_sentence = preprocess_sentence(' '.join([w.lemma for s in NLPL(sentence).sentences for w in s.words]), word_index)
# Predict sentiment
lemma_pred = model.predict(lemma_sentence)

# Output the result
print(f"Prediction: {lemma_pred[0][0]}")

if lemma_pred[0][0] > 0.5:
    print("Sentiment: Positive")
else:
    print("Sentiment: Negative")

2024-06-12 13:47:41.413206: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: indices[0,191] = 10451 is not in [0, 10000)
	 [[{{node sequential_4_1/embedding_1/GatherV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_4_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/noah/Documents/disability_trends/venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/noah/Documents/disability_trends/venv/lib64/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib64/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib64/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib64/python3.12/asyncio/events.py", line 88, in _ru